In [92]:
import requests
import json
import os
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

True

In [93]:
base_url = "https://jira.eficode.com"
access_token = os.getenv('JIRA_API_TOKEN')


In [94]:
url = f'{base_url}/rest/tempo-planning/1/allocation'
header = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}
res = requests.get(url=url, headers=header)

print(res.status_code)
data = res.json()

200


In [95]:
df = pd.DataFrame(columns=['plan_id', 'employee', 'start', 'end', 'hours_day', 'employee_day'])


for plan in data:
    row = {'plan_id': plan['id'], 'employee': plan['assignee']['key'], 'start': plan['start'], 'end': plan['end'], 'hours_day': int(plan['secondsPerDay'])/(60*60)}
    df.loc[len(df)] = row

df['start'] = pd.to_datetime(df['start'], format='%Y/%m/%d')
df['end'] = pd.to_datetime(df['end'], format='%Y/%m/%d')

# Drop user '1'
value_to_drop = '1'
df = df[df['employee'] != value_to_drop]
# dfgroup = df

# df

In [96]:
df['date_range'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('date_range')

df_summed = df_exploded.groupby(['date_range', 'employee'])['hours_day'].sum().reset_index()
# df_summed
plans = df_exploded.groupby(['date_range', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()
# Merge the new data frame with the employee data
merged_data = pd.merge(df_summed, plans, how='left')
# Read in the date table
date_table = pd.read_csv('Datetable.csv', parse_dates=['Date'])
# Convert the date column to a datetime object
date_table['Date'] = pd.to_datetime(date_table['Date'], format='%Y/%m/%d')

# date_table.set_index('Date', inplace=True)


df['Date'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('Date')

df_hours_sum = df_exploded.groupby(['Date', 'employee'])['hours_day'].sum().reset_index()
df_plan_agg = df_exploded.groupby(['Date', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()

df_merged = pd.merge(df_hours_sum, df_plan_agg, on=['Date', 'employee'])


In [97]:

begin = '1/1/2022'
end = '31/12/2024'

all_dates = pd.date_range(start=begin, end=end, freq='D')
all_names = df_merged['employee'].unique()
name_date_df = pd.MultiIndex.from_product([all_names, all_dates], names=['employee', 'Date']).to_frame(index=False)

df_all_days = pd.merge(name_date_df, df_merged, on=['employee', 'Date'], how='left').fillna({'hours_day': 0})

df_all_days['Date'] = df_all_days['Date'].dt.floor('D')
# df_all_days[df_all_days['employee'] == 'tuomas.kara']


merged_data = df_all_days.copy()

# merged_data.set_index('Date', inplace=True)

merged_data['allocation'] = 'Not fully allocated'

merged_data['is_workday'] = date_table['is_workday']

merged_data.loc[merged_data['is_workday'] == False, 'allocation'] = 'Not working day'

merged_data.loc[merged_data['hours_day'] >= 7.5, 'allocation'] = 'Fully allocated'
merged_data.loc[merged_data['hours_day'] <= 0.0, 'allocation'] = 'Not allocated'

merged_data.loc[merged_data['is_workday'] == False, 'hours_day'] = 0


merged_data = merged_data.merge(date_table[['Date', 'Year', 'Month', 'Week', 'Day']])



C:\Users\JariKaijansinkko\AppData\Local\Temp\ipykernel_12864\451153126.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  all_dates = pd.date_range(start=begin, end=end, freq='D')


In [98]:
merged_data.to_csv('updated_employee_data.csv', index=False)
